# 1. Importación y carga de Datos


In [22]:
#Importación de librerias
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix, plot_confusion_matrix, ConfusionMatrixDisplay
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
import pandas as pd
import numpy as np
import inflect
from sklearn.base import BaseEstimator, TransformerMixin, ClassifierMixin
from sklearn.compose import ColumnTransformer

# Importar/ Exportar modelos
from joblib import dump, load

#NLP

from nltk.tokenize import sent_tokenize, word_tokenize, RegexpTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
from nltk.sentiment.vader import SentimentIntensityAnalyzer

import matplotlib.pyplot as plt

In [2]:
!pip install inflect

     ---------------------------------------- 2.1/2.1 MB 1.8 MB/s eta 0:00:00


In [7]:
!pip install nltk
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')

     ---------------------------------------- 1.5/1.5 MB 5.6 MB/s eta 0:00:00
     -------------------------------------- 268.0/268.0 kB 5.5 MB/s eta 0:00:00


[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\kevin\AppData\Roaming\nltk_data...
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\kevin\AppData\Roaming\nltk_data...


True

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [23]:
np.random.seed(500)

In [24]:
#Carga de datos
df_original=pd.read_csv('SuicidiosProyecto.csv', sep=',', encoding = 'utf-8', index_col=0)
df_suicide = df_original.copy()

In [12]:
df_suicide.head(50)

,text,class
Unnamed: 0,,
173271,i want to destroy myselffor once everything wa...,suicide
336321,I kinda got behind schedule with learning for ...,non-suicide
256637,I'm just not sure anymoreFirst and foremost: I...,suicide
303772,please give me a reason to liveThats too much ...,suicide
293747,27f struggling to find meaning moving forwardI...,suicide
205651,Let’s get this bread 😎 Anyone know any good ba...,non-suicide
97174,"Day 126 of posting random ""fun"" facts everyday...",non-suicide
195945,Little brother is self mutilating. Please help...,suicide
305273,Why do women always go in groups to their wash...,non-suicide


In [13]:
#Información del DataFrame
df_suicide.describe()

,text,class
count,195700,195700
unique,195700,2
top,i want to destroy myselffor once everything wa...,non-suicide
freq,1,110165


In [9]:
df_suicide.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 195700 entries, 173271 to 305170
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    195700 non-null  object
 1   class   195700 non-null  object
dtypes: object(2)
memory usage: 4.5+ MB


# 2. Preprocesamiento y preparación de datos

Vamos a definir diferentes funciones para que puedan ser condensadas en transformadores que luego serán interpretados por el Pipeline.
El código se encuentra comentado dado que tuvimos que realizar estas clases en un archivo a parte para que el joblib que contiene el modelo pudiese cargarse de manera apropiada en la aplicación. Este archivo en cuestión se llama "cleaners.py"

In [25]:
#Conversión a numérico de la etiqueta
def class_sui(text):
  if text == 'suicide':
    return 1
  else:
    return 0

df_suicide['class'] = df_suicide['class'].apply(class_sui)

"""#Pasamos a minuscula
def minuscula(text):
  texto =[]
  for word in text:
    textos= word.lower()
    texto.append(textos)
  listToStr = ''.join([str(elem) for elem in texto]) 
  return listToStr  

#Eliminamos los números
def numeros(text):
  x=inflect.engine()
  texto = []
  for word in text :
      word.replace(" ","")
      if word.isdigit():
         textos=x.number_to_words(word)
         texto.append(textos)
      else:
         texto.append(word)    
  return texto   

def preprocesamiento(text):
  texto = minuscula(text)
  #texto = numeros(texto)
  return texto
"""
df_suicide.head(10)

,text,class
Unnamed: 0,,
173271,i want to destroy myselffor once everything wa...,1
336321,I kinda got behind schedule with learning for ...,0
256637,I'm just not sure anymoreFirst and foremost: I...,1
303772,please give me a reason to liveThats too much ...,1
293747,27f struggling to find meaning moving forwardI...,1
205651,Let’s get this bread 😎 Anyone know any good ba...,0
97174,"Day 126 of posting random ""fun"" facts everyday...",0
195945,Little brother is self mutilating. Please help...,1
305273,Why do women always go in groups to their wash...,0


Definimos las funciones para la tokenización y limpieza de datos final.
El código se encuentra comentado dado que tuvimos que realizar estas clases en un archivo a parte para que el joblib que contiene el modelo pudiese cargarse de manera apropiada en la aplicación. Este archivo en cuestión se llama "cleaners.py"

In [15]:
"""def limpieza_tokens(df):
  tokenizer = RegexpTokenizer(r'\w+')
  lemmatizer = WordNetLemmatizer()
  list_token = []
  for text in df['text']:
      # tokenize it 
      result = []
      results = tokenizer.tokenize(text)
      for word in results:
          # lemmatize it 
          words = lemmatizer.lemmatize(word)
          result.append(words)
      list_token.append(result)

  #Lista de stopwords
  count_vec = CountVectorizer(input='content', stop_words='english')
  stopw = set(count_vec.get_stop_words())

  #Eliminamos artículos, conjunciones, preposiciones, etc

  for a in range(0,len(list_token)-1):
    borrables = []
    for b in range(0,len(list_token[a])-1):
      if list_token[a][b] == 'm' or list_token[a][b] == 's' or list_token[a][b] == 've' or list_token[a][b] == 'don' or list_token[a][b] == 't' or list_token[a][b] == 'd' or list_token[a][b] == 'll' or list_token[a][b] in stopw:
        borrables.append(list_token[a][b])
    for c in borrables:
      list_token[a].remove(c)

  lst = []
  for i in list_token:
      listToStr = ' '.join([str(elem) for elem in i]) 
      lst.append(listToStr)
  return lst"""

Ahora creamos las clases que nos servirán para que el Pipeline transforme los datos apropiadamente.
El código se encuentra comentado dado que tuvimos que realizar estas clases en un archivo a parte para que el joblib que contiene el modelo pudiese cargarse de manera apropiada en la aplicación. Este archivo en cuestión se llama "cleaners.py"

In [26]:
"""#Clase de la transformación del texto
class Texto_Basico(BaseEstimator,TransformerMixin):
    def __init__(self):
        pass

    def fit(self,X,y=None):
        return self
    
    def transform(self,X,y=None):
        X = pd.DataFrame(X).copy()
        X['text'] = X['text'].apply(preprocesamiento)
        return X"""
from cleaners import Texto_Basico

In [27]:
"""#Clase de la tokenización y retoques finales
class Texto_Definitivo(BaseEstimator,TransformerMixin):
    def __init__(self):
        pass

    def fit(self,X,y=None):
        return self
    
    def transform(self,X,y=None):
        print(X.head(10))
        X = pd.DataFrame(X).copy()
        X['text'] = limpieza_tokens(X)
        print(X.info())
        return X
"""
from cleaners import Texto_Definitivo

# 3. Creación del Pipeline

In [28]:
#Pasos que ha de seguir el pipeline
transformer = ColumnTransformer(
    [('vec', TfidfVectorizer(), 'text')],   # column should be a string or int
    remainder='passthrough'
)

pre = [('texto', Texto_Basico()),
      ('token_y_mas', Texto_Definitivo()),
      ('tfidf_vectorizer', transformer),
      ('model', naive_bayes.MultinomialNB())]

In [29]:
# Se ajusta el pipeline a los datos
X = df_suicide.drop('class',axis=1)
y = df_suicide['class']

pipeline = Pipeline(pre)
pipeline = pipeline.fit(X,y)

                                                         text
Unnamed: 0                                                   
173271      i want to destroy myselffor once everything wa...
336321      i kinda got behind schedule with learning for ...
256637      i'm just not sure anymorefirst and foremost: i...
303772      please give me a reason to livethats too much ...
293747      27f struggling to find meaning moving forwardi...
205651      let’s get this bread 😎 anyone know any good ba...
97174       day 126 of posting random "fun" facts everyday...
195945      little brother is self mutilating. please help...
305273      why do women always go in groups to their wash...
69929       did you guys know that there's no school for g...
<class 'pandas.core.frame.DataFrame'>
Int64Index: 195700 entries, 173271 to 305170
Data columns (total 1 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    195700 non-null  object
dtypes: object(1)
memory usage

In [31]:
pipeline.score(X,y)

                                                         text
Unnamed: 0                                                   
173271      i want to destroy myselffor once everything wa...
336321      i kinda got behind schedule with learning for ...
256637      i'm just not sure anymorefirst and foremost: i...
303772      please give me a reason to livethats too much ...
293747      27f struggling to find meaning moving forwardi...
205651      let’s get this bread 😎 anyone know any good ba...
97174       day 126 of posting random "fun" facts everyday...
195945      little brother is self mutilating. please help...
305273      why do women always go in groups to their wash...
69929       did you guys know that there's no school for g...
<class 'pandas.core.frame.DataFrame'>
Int64Index: 195700 entries, 173271 to 305170
Data columns (total 1 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    195700 non-null  object
dtypes: object(1)
memory usage

0.9154215636177824

In [30]:
#Usamos la libreria joblib
filename = 'Proyecto1-Parte2Pro.joblib'

dump(pipeline, filename)

['Proyecto1-Parte2Pro.joblib']